In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 29 06:26:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   70C    P0   373W / 400W |  34881MiB / 40960MiB |    100%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [66]:
# !pip install -q -U sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaTokenizerFast, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

# model_id = "decapoda-research/llama-7b-hf"
# model_id = "huggyllama/llama-7b"
model_id = "EleutherAI/pythia-12b"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer = LlamaTokenizer.from_pretrained(model_id)
# tokenizer = LlamaTokenizer.from_pretrained(model_id)
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=8, 
#     lora_alpha=32, 
#     target_modules=["query_key_value"], 
#     lora_dropout=0.05, 
#     bias="none", 
#     task_type="CAUSAL_LM"
# )
config = LoraConfig.from_pretrained("../output-qlora")

model = get_peft_model(model, config)
print_trainable_parameters(model)

ValueError: Can't find 'adapter_config.json' at '../output-qlora'

In [11]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

print(data)

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-f1d1dd25192e24f2.arrow


DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2508
    })
})


In [41]:
from datasets import load_dataset

# Specify the path to your .jsonl file
path_to_jsonl_file = '../data/edie-v1/train.jsonl'

# Load your dataset
dataset = load_dataset('json', data_files={ 'train': path_to_jsonl_file })
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/json/default-e4a649f5e77c6bc7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 341
    })
})


In [63]:
tokenizer

dataset['train']

# sources = [example['input'] for example in dataset['train']]
# targets = [f"{example['output']}{tokenizer.eos_token}" for example in dataset['train']]
sources = [example['prompt'] for example in dataset['train']]
targets = [f"{example['completion']}{tokenizer.eos_token}" for example in dataset['train']]

print(sources[0])
print(targets[0])

I reached over and used my thumb to start my playlist, and as nymphetamine girl began, I let the mix of operatic and goth metal cause my body to begin to slowly relax, I reacted. I felt my muscles become smoother and time slowed down. I could suddenly see, almost like a premonition, everything that was going to happen as the pandora add played. The instant that it changed, to loco by coal chamber, time went from almost standing still, to life being in overdrive, and that's when my adrenal glands suddenly decided to kick in. No matter how hard my opponent hits me, it doesn't matter; I've never felt as alive as I have in the last two thousand years.

Writing Editor Feedback:

Good
Medium
COMMENT:
The opening scene is engaging and does a good job of setting the mood for the story. I like how music is used to enhance the experience and create a sense of anticipation for what's to come.
WHERE START:
I reached over and
WHERE END:
 slowly relax,
COMMENT END
Suggest
High
COMMENT:
Consider prov

In [64]:
max_len = 1536
tokenized_sources = tokenizer(
    # sources[:3],
    sources,
    # max_length=max_len,
    truncation=False,
)
# print(tokenized_sources)
for index, item in enumerate(tokenized_sources['input_ids']):
    if len(item) >= max_len:
        print(index, len(item))
# print(len(tokenized_sources['input_ids'][0]))

18 1777
21 1685
28 2106
29 1863
42 1687
44 2010
45 1622
48 1748
49 1582
58 1761
62 1655
64 1575
68 1947
89 1657
115 1906
132 1560
143 1934
144 1658
154 1852
155 2116
159 1587
165 1784
173 2027
176 1854
178 1588
189 1561
193 1815
196 1672
202 1594
223 1663
232 1891
241 1803
253 1567
255 1851
267 1981
268 1576
270 1555
273 1870
292 1598
305 1719
311 1579
334 1763
339 1572


In [29]:
# Load the model.
# Note: It can take a while to download LLaMA and add the adapter modules.
# You can also use the 13B model by loading in 4bits.

import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "huggyllama/llama-7b"
adapters_name = './output-edie-v1.0.1/checkpoint-30/adapter_model'

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model huggyllama/llama-7b into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB (GPU 0; 39.56 GiB total capacity; 7.37 GiB already allocated; 158.56 MiB free; 7.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF